<a href="https://colab.research.google.com/github/kevinOriginal/news-recommender/blob/main/Copy_of_Big_data_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from glob import glob
import pandas as pd
import pyarrow.parquet as pq

In [ ]:
import os, sys
import pandas as pd 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import torch
import torch.nn as nn
import numpy as np

In [ ]:
# 구글 드라이브 사용 시 주석 해제

from google.colab import drive
drive.mount('/content/mnt',force_remount=True)




Mounted at /content/mnt


In [ ]:
# Path에, 구글드라이브 내의 데이터 경로 설정
path = '/content/mnt/MyDrive/colab_data/big_data/user_vector.parquet'


In [ ]:
# GPU 할당 설정
GPU_NUM = 0 # 원하는 GPU 번호 입력
device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available() else 'cpu')
'''
torch.cuda.set_device(device) # change allocation of current GPU
print ('Current cuda device ', torch.cuda.current_device()) # check

# Additional Infos
if device.type == 'cuda':
    print(torch.cuda.get_device_name(GPU_NUM))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(GPU_NUM)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(GPU_NUM)/1024**3,1), 'GB')
'''


"\ntorch.cuda.set_device(device) # change allocation of current GPU\nprint ('Current cuda device ', torch.cuda.current_device()) # check\n\n# Additional Infos\nif device.type == 'cuda':\n    print(torch.cuda.get_device_name(GPU_NUM))\n    print('Memory Usage:')\n    print('Allocated:', round(torch.cuda.memory_allocated(GPU_NUM)/1024**3,1), 'GB')\n    print('Cached:   ', round(torch.cuda.memory_cached(GPU_NUM)/1024**3,1), 'GB')\n"

In [ ]:
# Read user_vector.parquet
'''
# 로컬 컴퓨터 실행시, 데이터 경로 설정
path = "./data/user_vector.parquet"
'''
df = pd.read_parquet(path) 

In [ ]:
#device = torch.device('cpu')
# Convert to torch 데이터를 텐서로 불러옴. cpu에 올리는 이유는 램 부족 때문
data = torch.DoubleTensor(df.values).to('cpu')
print("전체 데이터 : ", data.shape)

전체 데이터 :  torch.Size([356295, 1422])


In [ ]:
# 메인 유저 설정 (추천 받는 사용자)

main_user = data[7].to(device)

In [ ]:
# 비슷한 사용자를 몇 명 뽑을 것인지.

num_k = 500

In [ ]:
# Cosine Similarity 
cos = nn.CosineSimilarity(dim= -1, eps=1e-6)

# 높은 Cos값, 인덱스 저장할 리스트
stored_index = []
stored_values = []

# VRAM 부족으로, 10만개씩 끊어서 계산
# 10만개 당 cos_similarity 높은 num_k개씩 뽑아서 저장
for i in range(0, int(data.shape[0]/100000)+1) : 
    torch.cuda.empty_cache()
    val = cos(main_user,data[100000 * i : 100000 *i + 100000].to(device))
    values, index = torch.topk(val,num_k)
    index = index + 100000*i
    stored_index.append(index)
    stored_values.append(values)
    torch.cuda.empty_cache()

### 최종적으로 num_k개 유저 추리기
#
# 텐서로 변환
stored_index = torch.cat(stored_index,0)
stored_values = torch.cat(stored_values,0)
#print(stored_index,stored_values)

# 상위 5개 추리기
similar_values, index_of_stored_index = torch.topk(stored_values,num_k)

# 인덱스 변환
similar_index = []
for i in index_of_stored_index :
    #print(i)
    similar_index.append(stored_index[i])
### 최종적으로 num_k개 유저 추리기 끝

In [ ]:
# num_k개 추린 유저들의 벡터를 similar_users에 저장
similar_users = []
for i in range(0,num_k) :
  user_id = df.index[similar_index[i]]
  similar_value = similar_values[i]
  similar_users.append(df.loc[user_id])

  print("Index : {:10d}, ID : {:10s}, Similar Value : {}".format(similar_index[i],user_id,similar_value))

similar_users = torch.DoubleTensor(similar_users)

Index :          7, ID : 00004p2pq , Similar Value : 1.0
Index :     177914, ID : knig2u7aR , Similar Value : 0.4743416490252569
Index :     326818, ID : watc5I1EQ , Similar Value : 0.4743416490252569
Index :     231119, ID : noperCXP  , Similar Value : 0.4743416490252569
Index :     244744, ID : piet16WAu , Similar Value : 0.4472135954999579
Index :     284671, ID : siny7NeOG , Similar Value : 0.4472135954999579
Index :     224252, ID : nam5Vgf2  , Similar Value : 0.4472135954999579
Index :      74391, ID : dpem3RPfE , Similar Value : 0.4472135954999579
Index :      58180, ID : cwin25l1b , Similar Value : 0.4472135954999579
Index :     169183, ID : kimj2BtNU , Similar Value : 0.4472135954999579
Index :     274928, ID : sdh99n9cB , Similar Value : 0.4472135954999579
Index :     335035, ID : wonuvKQU  , Similar Value : 0.4472135954999579
Index :     192317, ID : leg55Xo30 , Similar Value : 0.4472135954999579
Index :     190350, ID : leec1P3vv , Similar Value : 0.4472135954999579
Index :

In [ ]:
# 벡터 체크
similar_users.shape

torch.Size([500, 1422])

In [ ]:
# num_k명의 벡터 평균치 계산
similar_users_mean = torch.mean(similar_users,dim=0)

# 하나의 평균 벡터가 나옴
similar_users_mean.shape

torch.Size([1422])

In [ ]:
# num_k명의 평균 벡터와, 주인공 유저와의 벡터 차잇값을 계산
diff_val = similar_users_mean - main_user.cpu()

In [ ]:
# 차이가 가장 많이 나는 값(뉴스) 추출. 차잇값, 인덱스 반환
num_recommend_news = 10 # 추천할 뉴스의 갯수
recommend_values, recommend_news_index = torch.topk(diff_val,num_recommend_news)

In [ ]:
# 추출된 인덱스에 해당하는 뉴스는, 
# 주인공 사용자는 보지 않았지만 비슷한 사용자들은 본 뉴스들이다.
# 혹은, 주인공 사용자만 보고 비슷한 사용자들은 전부 보지 않은 뉴스들이다.

print("추천도 : {}".format(recommend_values))
print("추천 뉴스의 data에서의 Columns 인덱스 : {}".format(recommend_news_index))

추천도 : tensor([0.0080, 0.0080, 0.0040, 0.0040, 0.0040, 0.0020, 0.0020, 0.0020, 0.0020,
        0.0020], dtype=torch.float64)
추천 뉴스의 data에서의 Columns 인덱스 : tensor([ 605, 1357,  636,  154,  384,  132,  145,  126,  115,  113])


In [ ]:
base_address = "https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid={}&aid={}"
# 추천 뉴스 출력
for i in range(0, num_recommend_news) :

  ids = df.columns[recommend_news_index[i].item()]
  oid = ids[4:7]
  aid = ids[-10:]
  #print(oid,aid)
  print (base_address.format(oid,aid),"     추천도 : {}".format(recommend_values[i]))

https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=023&aid=0003522820      추천도 : 0.008
https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=437&aid=0000235778      추천도 : 0.008
https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=023&aid=0003523196      추천도 : 0.004
https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=005&aid=0001310779      추천도 : 0.004
https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=015&aid=0004322636      추천도 : 0.004
https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=005&aid=0001310251      추천도 : 0.002
https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=005&aid=0001310498      추천도 : 0.002
https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=005&aid=0001309982      추천도 : 0.002
https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=005&aid=0001309120      추천도 : 0.002
https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=005&ai

In [ ]:
# 참고. 유저가 본 뉴스 목록들
# user_index에 유저 인덱스 값을 넣으면 된다
user_index = 9
test_user_vector = torch.DoubleTensor(data[user_index])
for i in range(0, test_user_vector.shape[0]) :
  if test_user_vector[i] > 0 :
    ids = df.columns[i]
    oid = ids[4:7]
    aid = ids[-10:]
    #print(oid,aid)
    print (base_address.format(oid,aid))
    

https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=001&aid=0011540279
https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=008&aid=0004393675
https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=015&aid=0004322277
https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=025&aid=0002992100
https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=055&aid=0000806872
https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=055&aid=0000807167
https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=081&aid=0003081123
https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=081&aid=0003081639
https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=119&aid=0002395475
https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=214&aid=0001030081


In [ ]:
import plotly as py
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
import plotly.io as pio

In [ ]:
pio.templates.default = "plotly_dark"
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.array(range(num_k)),y=similar_values.cpu().numpy(), name='cosine distance'))
fig.update_layout(
    autosize=False,
    width=500,
    height=500,
    margin=dict(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    )
)
fig.show()

In [ ]:
print(similar_users[1])
similar_users.shape

tensor([0., 0., 0.,  ..., 0., 0., 0.], dtype=torch.float64)


torch.Size([500, 1422])

In [ ]:
from sklearn import datasets
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

In [ ]:
scaler = StandardScaler()
pca = PCA()
pipeline = make_pipeline(scaler,pca)
pipeline.fit(data)
yff = pca.explained_variance_
pio.templates.default = "plotly_dark"
fig = go.Figure([go.Bar(x=np.array(range(pca.n_components_)), y=yff)])
fig.show()
#y값이 높은 feature일수록 차원을 줄일때 살려야하는 중요한 feature라는 뜻

In [ ]:
model = PCA(n_components=3)
pca_features = model.fit_transform(data)
xf = pca_features[:,0]
yf = pca_features[:,1]
zf = pca_features[:,2]
fig = go.Figure(data=[go.Scatter3d(x=xf, y=yf, z=zf,
                                   mode='markers', marker_size =5, name='Other User Vector')])
fig.add_scatter3d(connectgaps=True, x=[xf[0]], y=[yf[0]], z=[zf[0]], mode='markers', marker_size=5, name='Our User A Vector')
fig.add_scatter3d(connectgaps=True, x=xf[1:6], y=yf[1:6],z=zf[1:6], mode='markers', marker_size=5, name='selected User Vector') #초록점을 표시하는 라인이라서 주석처리 가능
fig.show()
#파란 점:표현한 벡터
#빨간 점:기준user의 벡터
#초록 점:비슷하다고 판단한 User의 벡터
#역시 생각처럼 예쁘게 나오지는 않는다(PCA과정에서 데이터가 소실되고 고차원의 데이터를 3차원으로 줄이는 데에는 한계가 있기 때문)
#하지만 초록점을 일부러 따로 표시하지 않는다면 벡터를 가시적으로 볼 수 있어서 더 좋은것같다. 
#plotly는 반응형 차트이기 때문에 휠이나 드래그로 줌인/줌아웃, 차트모델 돌려보기를 할 수 있습니다 